In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import getpass
import pandas as pd

env = 'TEST'

if env=='DEV':
    # Static connection parameters
    HOST = "theory.bcgov"  # Update as needed
    PORT = "5433"       # Default PostgreSQL port
    DATABASE = "odsdev"  # Update with your database name
elif env=='TEST':
    # Static connection parameters
    HOST = "theory.bcgov"  # Update as needed
    PORT = "5434"       # Default PostgreSQL port
    DATABASE = "odstst"  # Update with your database name

# Prompt for user credentials
username = 'bcts_etl_user'
password = getpass.getpass("Enter PostgreSQL password: ")

# Create a connection URL
connection_url = f"postgresql+psycopg2://{username}:{password}@{HOST}:{PORT}/{DATABASE}"




Enter PostgreSQL password: ········


In [2]:
def set_stage_analyst_role(schema, pi):

    if schema == 'BCTS_STAGING' and pi == 'No PI':
        return 'Read'
    else:
        return 'Deny'
    
def set_stage_analyst_pi_role(schema, pi):
    
    if schema == 'BCTS_STAGING' and pi == 'Audit':
        return 'Read'
    else:
        return 'Deny'
    
def set_analyst_role(schema, pi):
    
    if schema == 'BCTS_REPORTING' and pi == 'No PI':
        return 'Read'
    else:
        return 'Deny'
    
def set_analyst_pi_role(schema, pi):
    
    if schema == 'BCTS_REPORTING' and pi == 'Audit':
        return 'Read'
    else:
        return 'Deny'

In [3]:
# Define the schemas to inspect
schemas = ['lrm_replication', 'bcts_staging', 'bcts_reporting', 'mofclient_replication', 'bctsadmin_replication']

# Define query to get tables, views, and materialized views
query = """
SELECT table_schema, table_name, table_type 
FROM information_schema.tables 
WHERE table_schema IN :schemas;
"""

try:
    # Create an SQLAlchemy engine
    engine = create_engine(connection_url)
    
    # Connect to the database
    with engine.connect() as connection:
        print("Connection successful!")
        
        # Execute the query
        result = connection.execute(text(query), {"schemas": tuple(schemas)})
        
        # Convert result to DataFrame
        df = pd.DataFrame(result.fetchall(), columns=['SCHEMA', 'TABLE_NAME', 'TYPE'])
        df['SCHEMA'] = df['SCHEMA'].apply(lambda x: x.upper())
        df['TABLE_NAME'] = df['TABLE_NAME'].apply(lambda x: x.upper())
        
        # Save to Excel


except SQLAlchemyError as e:
    print(f"An error occurred: {e}")

# df_ex = pd.read_csv('./BCTS_ACCESS_CATALOG.csv')
dfo = df.copy()
dfo = dfo.sort_values(['SCHEMA', 'TABLE_NAME'])
dfo['KEY'] = list(range(1, len(dfo) + 1))
dfo['AUDIT_FOR_PI'] = 'No PI'
dfo['BCTS_DEV_ROLE'] = 'Read'

dfo['BCTS_STAGE_ANALYST_ROLE'] = dfo.apply(lambda row: set_stage_analyst_role(row['SCHEMA'], row['AUDIT_FOR_PI']), axis=1)
dfo['BCTS_STAGE_ANALYST_PI_ROLE'] = dfo.apply(lambda row: set_stage_analyst_pi_role(row['SCHEMA'], row['AUDIT_FOR_PI']), axis=1)

dfo['BCTS_ANALYST_ROLE'] = dfo.apply(lambda row: set_analyst_role(row['SCHEMA'], row['AUDIT_FOR_PI']), axis=1)
dfo['BCTS_ANALYST_PI_ROLE'] = dfo.apply(lambda row: set_analyst_pi_role(row['SCHEMA'], row['AUDIT_FOR_PI']), axis=1)
dfo = dfo[['KEY',
          'SCHEMA',
         'TABLE_NAME',
         'TYPE',
         'AUDIT_FOR_PI',
         'BCTS_DEV_ROLE',
         'BCTS_STAGE_ANALYST_ROLE',
         'BCTS_STAGE_ANALYST_PI_ROLE',
         'BCTS_ANALYST_ROLE',
         'BCTS_ANALYST_PI_ROLE']]
dfo.to_excel("schema_details.xlsx", index=False)
print("Excel file 'schema_details.xlsx' saved successfully.")

Connection successful!
Excel file 'schema_details.xlsx' saved successfully.


In [66]:
dfo

,KEY,SCHEMA,TABLE_NAME,TYPE,AUDIT_FOR_PI,BCTS_DEV_ROLE,BCTS_STAGE_ANALYST_ROLE,BCTS_STAGE_ANALYST_PI_ROLE,BCTS_ANALYST_ROLE,BCTS_ANALYST_PI_ROLE
86,1,BCTSADMIN_REPLICATION,BCTS_TENURE_BIDDER,BASE TABLE,No PI,Read,Deny,Deny,Deny,Deny
40,2,BCTSADMIN_REPLICATION,BCTS_TIMBER_SALE,BASE TABLE,No PI,Read,Deny,Deny,Deny,Deny
65,3,BCTSADMIN_REPLICATION,NO_SALE_RATIONALE_CODE,BASE TABLE,No PI,Read,Deny,Deny,Deny,Deny
102,4,BCTS_REPORTING,ANNUAL_DEVELOPED_VOLUME,BASE TABLE,No PI,Read,Deny,Deny,Read,Deny
29,5,BCTS_REPORTING,CURRENTLY_IN_MARKET,BASE TABLE,No PI,Read,Deny,Deny,Read,Deny
...,...,...,...,...,...,...,...,...,...,...
35,110,LRM_REPLICATION,SUB_OPERATING_AREA,BASE TABLE,No PI,Read,Deny,Deny,Deny,Deny
94,111,LRM_REPLICATION,TENURE_TYPE,BASE TABLE,No PI,Read,Deny,Deny,Deny,Deny
108,112,LRM_REPLICATION,V_RES_VT_FDTM_TEAM,BASE TABLE,No PI,Read,Deny,Deny,Deny,Deny
2,113,MOFCLIENT_REPLICATION,ORG_UNIT,BASE TABLE,No PI,Read,Deny,Deny,Deny,Deny


In [78]:
x = \
"""
'COMMITMENTS',
 'PERMIT_ALLOCATION',
 'DIVISION',
 'COMMITMENT_PARTITION',
 'LICENCE_ALLOCATION',
 'CODE_LOOKUP',
 'CTOR_CONTRACTOR',
 'APPORTIONMENT',
 'MARK',
 'TENURE_TYPE',
 'V_LRM_COMMITMENTS',
 'BLOCK_ADMIN_ZONE',
 'MANAGEMENT_UNIT',
 'LICENCE',
 'BLOCK_ALLOCATION',
 'CUT_PERMIT',
 'BCTS_HARVEST_HISTORY',
 'LRM_VT_COMMIT_LIC_TYPE',
 'MARK_ALLOCATION',
 'CTOR_CONTRACTOR_LOCATION',
 'activity',
 'division',
 'licence_allocation',
 'licence',
 'activity_type',
 'management_unit',
 'tenure_type',
 'CODE_LOOKUP',
 'ctor_contractor',
 'activity_class',
 'CUT_BLOCK_SHAPE_EVW',
 'MANAGEMENT_UNIT',
 'LICENCE',
 'MARK',
 'CUT_PERMIT',
 'BLOCK_ALLOCATION',
 'DIVISION',
 'CUT_BLOCK'
"""

In [96]:
new = [x.replace("'", "")[:-1].strip().upper() for x in x.split('\n')[1:-1]]

In [100]:
ex = list(dfo.query('SCHEMA=="LRM_REPLICATION"')['TABLE_NAME'])

In [101]:
[x for x in new if x not in ex]

['V_LRM_COMMITMENTS', 'CUT_BLOCK_SHAPE_EVW', 'CUT_BLOC']

In [102]:
[x for x in new if x  in ex]

['COMMITMENTS',
 'PERMIT_ALLOCATION',
 'DIVISION',
 'COMMITMENT_PARTITION',
 'LICENCE_ALLOCATION',
 'CODE_LOOKUP',
 'CTOR_CONTRACTOR',
 'APPORTIONMENT',
 'MARK',
 'TENURE_TYPE',
 'BLOCK_ADMIN_ZONE',
 'MANAGEMENT_UNIT',
 'LICENCE',
 'BLOCK_ALLOCATION',
 'CUT_PERMIT',
 'BCTS_HARVEST_HISTORY',
 'LRM_VT_COMMIT_LIC_TYPE',
 'MARK_ALLOCATION',
 'CTOR_CONTRACTOR_LOCATION',
 'ACTIVITY',
 'DIVISION',
 'LICENCE_ALLOCATION',
 'LICENCE',
 'ACTIVITY_TYPE',
 'MANAGEMENT_UNIT',
 'TENURE_TYPE',
 'CODE_LOOKUP',
 'CTOR_CONTRACTOR',
 'ACTIVITY_CLASS',
 'MANAGEMENT_UNIT',
 'LICENCE',
 'MARK',
 'CUT_PERMIT',
 'BLOCK_ALLOCATION',
 'DIVISION']

In [103]:
len(new)

38